# Interactive Line Charts and Choropleth in Altair

Interactive Line Charts and Choropleth from data stored in a Pandas dataframe for WHO flu data

In [1]:
%matplotlib inline
import pandas as pd
import altair as alt
from vega_datasets import data

In [2]:
flunet_df = pd.read_csv('./data/flunet2010_11countries_106.csv')
flunet_df.head()

,week,Afghanistan,Argentina,Australia,Canada,China,Colombia,Egypt,Germany,Ireland,South Africa,USA
0,1,5.0,4,2,41,2179,36,739,26.0,23,0,366
1,2,13.0,21,1,15,2213,36,396,24.0,8,1,396
2,3,4.0,6,1,8,2228,14,192,18.0,4,0,447
3,4,0.0,1,0,14,2027,11,80,NaN,8,0,402
4,5,0.0,4,1,12,1813,8,56,NaN,4,0,404


In [3]:
countries = flunet_df.columns.tolist()[1:]
transformed_df = pd.melt(flunet_df, id_vars=['week'], value_vars=countries,var_name = 'country' ,value_name='flu_cases')

In [4]:
transformed_df['index'] = transformed_df['week']%53

In [5]:
transformed_df.head()

,week,country,flu_cases,index
0,1,Afghanistan,5.0,1
1,2,Afghanistan,13.0,2
2,3,Afghanistan,4.0,3
3,4,Afghanistan,0.0,4
4,5,Afghanistan,0.0,5


## Task 1

### Interactive line chart

Line chart showing flu cases per week
  - Based on the country
  - Linked to a control allowing selection of which country to be shown


Additionally, added multi-selector/checkbox instead of drop down for multiple countries.

#### Features
  - Interactive plot
  - Interactions based on country
  - Tooltips showing Exact Count of number of flu cases

In [6]:
def plot(selection):
    """
    """
    color = alt.condition(selection,
                      alt.Color('country:N', legend=None),
                      alt.value('lightgray'))

    base = alt.Chart(transformed_df,
              title="Flu cases per week").encode(
            x=alt.X('week:Q', title='Week'),
            y=alt.Y('flu_cases:Q',title='Flu Cases'),
            color=color ,
            tooltip=([alt.Tooltip('flu_cases:Q', title='Total Cases')])
    )

    line = base.mark_line().add_selection(
        selection
    ).interactive(bind_y=False)
    
    return line,color

In [7]:
selector = alt.selection(type='single',fields=['country'], 
                          bind=alt.binding_select(options=countries),
                          name='Select')

line,_ = plot(selector)
line

alt.Chart(...)

In [8]:
country = pd.DataFrame({'country':countries})
selection = alt.selection_multi(fields=['country'])

line,color = plot(selection)
make_selector = alt.Chart(country).mark_rect().encode(
                    y='country',
                    color=color).add_selection(
                    selection)
line | make_selector

alt.HConcatChart(...)

## Task 2

### Choropleth Map

Create a Choropleth Map
  - Showing Flu Cases per Country for whole year 

In [9]:
country_data = pd.read_csv('./data/countries.csv')
name_id = country_data[country_data['name'].isin(countries)][['name', 'id']]

In [10]:
choropleth_df = name_id.reset_index(drop=True).merge(
                    pd.DataFrame(flunet_df.sum().T),left_on='name',right_index=True).rename(columns={0: "flu_cases"})

In [11]:
choropleth_df

,name,id,flu_cases
0,Afghanistan,4,26.0
1,Argentina,32,2115.0
2,Australia,36,1252.0
3,Canada,124,3978.0
4,China,156,44166.0
5,Colombia,170,786.0
6,Egypt,818,3028.0
7,Germany,276,189.0
8,Ireland,372,661.0
9,South Africa,710,1333.0


In [12]:
source = alt.topo_feature(data.world_110m.url, 'countries')

background = alt.Chart(source).mark_geoshape(fill='lightgray',
    stroke='white'
).project("equirectangular")

choropleth = alt.Chart(source).mark_geoshape(
    stroke='white'
).transform_lookup(
    lookup='id',
    from_=alt.LookupData(choropleth_df, 'id', list(choropleth_df.columns)[2:])
).encode(
    color="flu_cases:Q",
    tooltip=([alt.Tooltip('flu_cases:Q', title='Total Cases')])
).properties(
    width=500,
    height=300,
    title="Annual Net Flu Cases"
).project("equirectangular")

background+choropleth

alt.LayerChart(...)